In [13]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.models import Sequential

import os

In [14]:
batch_size = 128
img_height = 180
img_width = 180
seed = 42

In [15]:
! rm -r ./data
! mkdir ./data
! tar -xf ./file.tgz -C ./data

In [16]:
image_size = (img_height, img_width)

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "./data",
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
)

Found 3000 files belonging to 2 classes.
Using 2400 files for training.
Using 600 files for validation.


In [17]:
class_names = train_ds.class_names
print(class_names)

['cats', 'dogs']


In [18]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [19]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,img_width,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [20]:
num_classes = len(class_names)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 647ms/step - accuracy: 0.5061 - loss: 1.1676 - val_accuracy: 0.5083 - val_loss: 0.6911
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 574ms/step - accuracy: 0.5313 - loss: 0.6879 - val_accuracy: 0.5817 - val_loss: 0.6757
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 507ms/step - accuracy: 0.6113 - loss: 0.6620 - val_accuracy: 0.6233 - val_loss: 0.6477
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - accuracy: 0.6638 - loss: 0.6201 - val_accuracy: 0.6400 - val_loss: 0.6447
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 505ms/step - accuracy: 0.6929 - loss: 0.5914 - val_accuracy: 0.6750 - val_loss: 0.6063
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 519ms/step - accuracy: 0.7033 - loss: 0.5793 - val_accuracy: 0.6533 - val_loss: 0.6456
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 543ms/step - accuracy: 0.7082 - loss: 0.5795 - val_accuracy: 0.6500 - val_loss: 0.6484
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 513ms/step - accuracy: 0.6985 - loss: 0.5692 - val_accu

In [ ]:
model.save_weights('./model.weights.h5')

In [ ]:
data_url = "file://" + os.path.expanduser("~/Downloads/test_cat2.jpg")
data_path = tf.keras.utils.get_file('test_cat2', origin=data_url)

img = tf.keras.utils.load_img(
    data_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
This image most likely belongs to dogs with a 65.71 percent confidence.
